In [5]:
print("helloworld")

helloworld


In [56]:
import numpy as np
import torch
import torch.nn as nn

# 1. 먼저 전체 데이터를 하나의 리스트로 읽기
users, movies, ratings = [], [], []
user_len, movie_len = 0, 0

with open("ml-1m/ratings.dat", "r") as f:
    for line in f:
        u, m, r, t = line.split("::")
        user_len = max(user_len, int(u))
        movie_len = max(movie_len, int(m))
        users.append(int(u))
        movies.append(int(m))
        ratings.append(float(r))

indices = np.arange(len(ratings))
np.random.shuffle(indices)

users = np.array(users)[indices]
movies = np.array(movies)[indices]
ratings = np.array(ratings)[indices]

device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")


users = torch.tensor(users, dtype=torch.long, device="mps")
movies = torch.tensor(movies, dtype=torch.long, device = "mps")
ratings = torch.tensor(ratings, dtype=torch.float, device = "mps")

# 3. 데이터 분할 (예: 60% 학습, 20% 검증, 20% 테스트)
n_samples = len(ratings)
train_size = int(0.6 * n_samples)
valid_size = int(0.2 * n_samples)

# 학습 데이터
users_train = users[:train_size]
movies_train = movies[:train_size]
ratings_train = ratings[:train_size]


# 검증 데이터
users_valid = users[train_size:train_size+valid_size]
movies_valid = movies[train_size:train_size+valid_size]
ratings_valid = ratings[train_size:train_size+valid_size]


# 테스트 데이터
users_test = users[train_size+valid_size:]
movies_test = movies[train_size+valid_size:]
ratings_test = ratings[train_size+valid_size:]






FileNotFoundError: [Errno 2] No such file or directory: 'ml-1m/ratings.dat'

In [7]:
from torch.utils.data import DataLoader, TensorDataset

train_loader = DataLoader(TensorDataset(users_train, movies_train, ratings_train), batch_size=64, shuffle=True)
valid_loader = DataLoader(TensorDataset(users_valid, movies_valid, ratings_valid), batch_size=64, shuffle=False)
test_loader = DataLoader(TensorDataset(users_test, movies_test, ratings_test), batch_size=64, shuffle=False)



torch.Size([200043])


# a + b_i + b_u + r_u*r_i

In [8]:
user_emb = nn.Embedding(user_len+1, 100)
item_emb = nn.Embedding(movie_len+1, 100)
user_bias = nn.Embedding(user_len+1, 1)
item_bias = nn.Embedding(movie_len+1, 1)
alpha = ratings.float().mean()
u1 = user_emb(torch.LongTensor([1]))
i1 = item_emb(torch.LongTensor([1]))
u_b = user_bias(torch.LongTensor([1]))
i_b = item_bias(torch.LongTensor([1]))
print(torch.sum(u1*i1))
print(torch.sum(u1*i1, dim=1))

tensor(5.3114, grad_fn=<SumBackward0>)
tensor([5.3114], grad_fn=<SumBackward1>)


In [7]:
import torch.nn as nn
import torch.optim as optim
import torch

class MF(nn.Module):
    def __init__(self, user_len, movie_len, ratings, rank):
        super(MF, self).__init__()
        self.user_len = user_len
        self.movie_len = movie_len
        self.rank = rank
        self.alpha = ratings.float().mean()
        self.user_bias = nn.Embedding(user_len+1, 1)
        self.item_bias = nn.Embedding(movie_len+1, 1)
        self.user_reps = nn.Embedding(user_len+1, rank)
        self.item_reps = nn.Embedding(movie_len+1, rank)
    def forward(self, user,item):
        user_bias = self.user_bias(user)
        item_bias = self.item_bias(item)
        user_emb = self.user_reps(user)
        item_emb = self.item_reps(item)
        return self.alpha + user_bias + item_bias + torch.sum(user_emb*item_emb)
    

    
    


In [14]:


device = torch.device("mps" if torch.mps.is_available() else "cpu")

rank = 100

epochs = 10

model = MF(user_len, movie_len, ratings, rank).to(device)

optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.MSELoss()



epoch_train_losses, epoch_val_losses = [], []

for epoch in range(epochs):
    train_losses , valid_losses = [],[]    
    model.train()
    for i, (user, movie, rating) in enumerate(zip(users_train, movies_train, ratings_train)):
        if(i%10000==0):
            print(f"{i}번째 입니다")
        rating = rating.unsqueeze(0)
        pred = model(user, movie)
        loss = criterion(pred, rating)
        train_losses.append(loss.item())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    model.eval()
    for user, movie, rating in zip(users_valid, movies_valid, ratings_valid):
        pred = model(user,movie)
        loss = criterion(pred, rating)
        valid_losses.append(loss.item())
    epoch_train_loss = np.mean(train_losses)
    epoch_val_loss = np.mean(valid_losses)
    epoch_train_losses.append(epoch_train_loss)
    epoch_val_losses.append(epoch_val_loss)

    print(f'Epoch: {epoch}, Train Loss: {epoch_train_loss:0.1f}, Val Loss:{epoch_val_loss:0.1f}')



0번째 입니다
10000번째 입니다
20000번째 입니다
30000번째 입니다
40000번째 입니다
50000번째 입니다
60000번째 입니다
70000번째 입니다
80000번째 입니다
90000번째 입니다
100000번째 입니다
110000번째 입니다
120000번째 입니다
130000번째 입니다
140000번째 입니다
150000번째 입니다
160000번째 입니다
170000번째 입니다
180000번째 입니다
190000번째 입니다
200000번째 입니다
210000번째 입니다
220000번째 입니다
230000번째 입니다
240000번째 입니다
250000번째 입니다
260000번째 입니다
270000번째 입니다
280000번째 입니다
290000번째 입니다
300000번째 입니다
310000번째 입니다
320000번째 입니다
330000번째 입니다
340000번째 입니다
350000번째 입니다
360000번째 입니다
370000번째 입니다
380000번째 입니다
390000번째 입니다
400000번째 입니다
410000번째 입니다
420000번째 입니다
430000번째 입니다
440000번째 입니다
450000번째 입니다
460000번째 입니다
470000번째 입니다
480000번째 입니다
490000번째 입니다
500000번째 입니다
510000번째 입니다
520000번째 입니다
530000번째 입니다
540000번째 입니다
550000번째 입니다
560000번째 입니다
570000번째 입니다
580000번째 입니다
590000번째 입니다
600000번째 입니다


/opt/homebrew/anaconda3/envs/dmlab/lib/python3.11/site-packages/torch/nn/modules/loss.py:610: UserWarning: Using a target size (torch.Size([])) that is different to the input size (torch.Size([1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch: 0, Train Loss: 45.9, Val Loss:17.7
0번째 입니다
10000번째 입니다
20000번째 입니다
30000번째 입니다
40000번째 입니다
50000번째 입니다
60000번째 입니다
70000번째 입니다
80000번째 입니다
90000번째 입니다
100000번째 입니다
110000번째 입니다
120000번째 입니다
130000번째 입니다
140000번째 입니다
150000번째 입니다
160000번째 입니다
170000번째 입니다
180000번째 입니다
190000번째 입니다
200000번째 입니다
210000번째 입니다
220000번째 입니다
230000번째 입니다
240000번째 입니다
250000번째 입니다
260000번째 입니다
270000번째 입니다
280000번째 입니다
290000번째 입니다
300000번째 입니다
310000번째 입니다
320000번째 입니다
330000번째 입니다
340000번째 입니다
350000번째 입니다
360000번째 입니다
370000번째 입니다
380000번째 입니다
390000번째 입니다
400000번째 입니다
410000번째 입니다
420000번째 입니다
430000번째 입니다
440000번째 입니다
450000번째 입니다
460000번째 입니다
470000번째 입니다
480000번째 입니다
490000번째 입니다
500000번째 입니다
510000번째 입니다
520000번째 입니다
530000번째 입니다
540000번째 입니다
550000번째 입니다
560000번째 입니다
570000번째 입니다
580000번째 입니다
590000번째 입니다
600000번째 입니다
Epoch: 1, Train Loss: 7.7, Val Loss:8.6
0번째 입니다
10000번째 입니다
20000번째 입니다
30000번째 입니다
40000번째 입니다
50000번째 입니다
60000번째 입니다
70000번째 입니다
80000번째 입니다
90000번째 입니다
100000번째 입니다
110000번째 입

In [19]:
model.eval()
with torch.no_grad():
    user = users_test[5].unsqueeze(0)
    movie = movies_test[5].unsqueeze(0)
    ratings = ratings_test[5].unsqueeze(0)
    pred = model(user, movie)
    print(pred)

tensor([[3.2708]], device='mps:0')


In [20]:
print(ratings)

tensor([3.], device='mps:0')


In [26]:
print(users_test.shape)
print(movies_test.shape)
print(ratings_test.shape)
ratings_test

torch.Size([200043])
torch.Size([200043])
torch.Size([200043])


tensor([5., 4., 3.,  ..., 4., 3., 5.], device='mps:0')

In [39]:
print(f"{users_test[4]}유저가 {movies_test[4]}에 {ratings_test[4]}점을 줬다")

1472유저가 2421에 3.0점을 줬다


In [38]:
model.eval()
with torch.no_grad():
    user = users_test[4].unsqueeze(0)
    movie = movies_test[4].unsqueeze(0)
    ratings = ratings_test[4].unsqueeze(0)
    pred = model(user, movie)
    print(f"{user}유저가 {movie}에 {pred}점을 줄 것이라 예측했다")

tensor([1472], device='mps:0')유저가 tensor([2421], device='mps:0')에 tensor([[2.0162]], device='mps:0')점을 줄 것이라 예측했다


In [16]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# 데이터 로드 및 처리
users, movies, ratings = [], [], []
user_len, movie_len = 0, 0

with open("data/ml-1m/ratings.dat", "r") as f:
    for line in f:
        u, m, r, t = line.split("::")
        user_len = max(user_len, int(u))
        movie_len = max(movie_len, int(m))
        users.append(int(u))
        movies.append(int(m))
        ratings.append(float(r))  # float 유지!

# 데이터 섞기
indices = np.arange(len(ratings))
np.random.shuffle(indices)

users = np.array(users)[indices]
movies = np.array(movies)[indices]
ratings = np.array(ratings)[indices]

device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

users = torch.tensor(users, dtype=torch.long, device=device)
movies = torch.tensor(movies, dtype=torch.long, device=device)
ratings = torch.tensor(ratings, dtype=torch.float, device=device)  # float로 변환

# 데이터 분할 (60% 학습, 20% 검증, 20% 테스트)
n_samples = len(ratings)
train_size = int(0.6 * n_samples)
valid_size = int(0.2 * n_samples)

users_train, movies_train, ratings_train = users[:train_size], movies[:train_size], ratings[:train_size]
users_valid, movies_valid, ratings_valid = users[train_size:train_size+valid_size], movies[train_size:train_size+valid_size], ratings[train_size:train_size+valid_size]
users_test, movies_test, ratings_test = users[train_size+valid_size:], movies[train_size+valid_size:], ratings[train_size+valid_size:]

# DataLoader 활용
batch_size = 64
train_data = DataLoader(TensorDataset(users_train, movies_train, ratings_train), batch_size=batch_size, shuffle=True)
valid_data = DataLoader(TensorDataset(users_valid, movies_valid, ratings_valid), batch_size=batch_size, shuffle=False)

# 모델 정의
class MF(nn.Module):
    def __init__(self, user_len, movie_len, rank):
        super(MF, self).__init__()
        self.user_bias = nn.Embedding(user_len+1, 1)
        self.item_bias = nn.Embedding(movie_len+1, 1)
        self.user_reps = nn.Embedding(user_len+1, rank)
        self.item_reps = nn.Embedding(movie_len+1, rank)

    def forward(self, user, item):
        user_bias = self.user_bias(user).squeeze()
        item_bias = self.item_bias(item).squeeze()
        user_emb = self.user_reps(user)
        item_emb = self.item_reps(item)
        return user_bias + item_bias + torch.sum(user_emb * item_emb, dim=1)

# 학습 설정
rank = 100
epochs = 10

model = MF(user_len, movie_len, rank).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.MSELoss()

# 학습 루프
for epoch in range(epochs):
    model.train()
    train_losses = []
    for user, movie, rating in train_data:
        user, movie, rating = user.to(device), movie.to(device), rating.to(device)

        pred = model(user, movie)
        loss = criterion(pred, rating)
        train_losses.append(loss.item())

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # 검증 루프
    model.eval()
    valid_losses = []
    with torch.no_grad():
        for user, movie, rating in valid_data:
            user, movie, rating = user.to(device), movie.to(device), rating.to(device)
            pred = model(user, movie)
            loss = criterion(pred, rating)
            valid_losses.append(loss.item())

    # 로그 출력
    print(f'Epoch: {epoch+1}, Train Loss: {np.mean(train_losses):.4f}, Val Loss: {np.mean(valid_losses):.4f}')


Epoch: 1, Train Loss: 89.8780, Val Loss: 66.8109
Epoch: 2, Train Loss: 40.3435, Val Loss: 39.0168
Epoch: 3, Train Loss: 17.4000, Val Loss: 24.1078
Epoch: 4, Train Loss: 7.6721, Val Loss: 16.4039
Epoch: 5, Train Loss: 3.7005, Val Loss: 12.3138
Epoch: 6, Train Loss: 2.0348, Val Loss: 9.9884
Epoch: 7, Train Loss: 1.3066, Val Loss: 8.6288
Epoch: 8, Train Loss: 0.9745, Val Loss: 7.7759
Epoch: 9, Train Loss: 0.8117, Val Loss: 7.2246
Epoch: 10, Train Loss: 0.7193, Val Loss: 6.8318


In [23]:
model.eval()
with torch.no_grad():
    user = users_test[3].unsqueeze(0)
    movie = movies_test[3].unsqueeze(0)
    ratings = ratings_test[3].unsqueeze(0)
    pred = model(user, movie)
    print(pred)

tensor([4.3802], device='mps:0')


In [24]:
print(ratings)

tensor([5.], device='mps:0')
